# Prerequisites
Install the `OpenPAI` sdk from `github` and specify your cluster information in `~/.openpai/clusters.json`. And for simplicity and security, we recommand user to setup necessary information in `.openpai/defaults.json` other than shown in the example notebook. (Refer to for [0-install-sdk-specify-openpai-cluster.ipynb]() more details.)

<font color=red>Please make sure you have set default values for ***cluster-alias***, ***image*** and ***workspace***.</font>

In [ ]:
from openpaisdk.command_line import Engine, get_client

clusters = Engine().process(['cluster', 'list'])
default_values = Engine().process(['default', 'list'])

assert ('cluster-alias' in default_values or len(clusters)==1) \
    and 'image' in default_values \
    and 'workspace' in default_values, \
    "Please setup necessary infromation"

# Prepare your script locally
We prepare an example to show how reuse the same code in local environment and in `OpenPAI` node.

In [ ]:
import os
os.makedirs('script', exist_ok=True)

In [ ]:
%%writefile script/test.py

import os
import argparse
from openpaisdk.core import in_job_container

def say_hi(name):
    if in_job_container():
        print('Hello, {}! I am in a container: {}'.format(name, os.environ['PAI_CONTAINER_ID']))
    else:
        print('Hello, {}! I am from local script: {}'.format(name, __file__))

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('name', help='your name')
    args = parser.parse_args()
    say_hi(args.name)

In [ ]:
! python script/test.py Alice

# Submit in the easiest way
For the jobs that are simple (e.g. with only one task role), the CLI tool provides a shortcut to combine create, task and submit into only one command `fast`. 

In [ ]:
from uuid import uuid4 as randstr
import shutil
job_name = 'test_new_cli_fast_' + randstr().hex
print("newly generated job name", job_name)

# shutil.rmtree('.openpai/jobs')  # uncomment to remove unused cache files

! opai job fast -j {job_name} -p simplejson python script/test.py Alice

# Submit to `OpenPAI` step by step
Now we will submit above command to `OpenPAI` through the `opai job` tool. 

Since `OpenPAI` doesnot allow duplicated job name, you must define a unique name with `-j <job-name>`. In below commands, we show how to create a job, add packages to install, add task role commands, and the submit it. 

To simplify user input, the `create` command would set current job as default (by setting default `job-name` to current), and the `submit` command would release the default (by removing `job-name` in default). To disable this, use `--dont-set-as-default` in `create`. 

In [ ]:
from uuid import uuid4 as randstr
job_name = 'test_new_cli_' + randstr().hex
print("newly generated job name", job_name)

! opai job new -j {job_name}
! opai task add -t main python script/test.py Alice
! opai job submit

By clicking the printed job link, you may find below line in the `STDOUT`

```
Hello, Alice! I am in a container: container_e26_1556418566879_0058_01_000002
```
<font color=red>_The auto fetch of command output is under development_</font>

# Deal with more complicated jobs
Here we will have a try with adding `requirements` (e.g. python packages to be installed, data mapping links) to the job. 

In [ ]:
from uuid import uuid4 as randstr
job_name = 'test_new_cli_' + randstr().hex
print("newly generated job name", job_name)

! opai job new -j {job_name}

! opai require pip simplejson
! opai require weblink http://proceedings.mlr.press/v80/bender18a/bender18a.pdf papers

! opai task add -t main ls papers

! opai job submit

# Query submitted jobs
You may use `opai job list` to query more informaiton about the job. 